## Load the sequence

In [5]:
!pip install -q minbpe

ERROR: Could not find a version that satisfies the requirement minbpe (from versions: none)
ERROR: No matching distribution found for minbpe


In [2]:
with open("model_llm/data/feynman_combined_text.txt", "r") as f:
    text_sequence = f.read()

len(text_sequence)

2027114

## BPE algorithm

I am using the [minBPE](https://github.com/karpathy/minbpe) repository to tokenize the sequence of text.

In [3]:
import sys
sys.path.append('..')

Start by training the tokenizer on the text sequence that you saved in the previous notebook.

In [7]:
from model_llm.minbpe import BasicTokenizer

tokenizer = BasicTokenizer()
tokenizer.train(text_sequence, vocab_size=1024)

  0%|          | 0/768 [00:00<?, ?it/s]

100%|██████████| 768/768 [04:55<00:00,  2.60it/s]


Visualize the vocabulary.

In [8]:
vocab = tokenizer.vocab
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

Test the tokenizer.

In [9]:
tokenizer.encode("action-reaction experiment")

[345, 367, 45, 275, 345, 375, 1023, 343]

In [10]:
tokenizer.decode([345, 367, 45, 275, 345, 375, 1023, 343])

'action-reaction experiment'

Add special tokens to the vocabulary. These tokens are going to be used a lot in the fine-tuning step.

In [11]:
max_vocab_id = list(tokenizer.vocab.keys())[-1]
tokenizer.special_tokens = {
    "<|startoftext|>": max_vocab_id + 1,
    "<|separator|>": max_vocab_id + 2,
    "<|endoftext|>": max_vocab_id + 3,
    "<|unk|>": max_vocab_id + 4
}

I have more than 618K tokens for training and validation. This is pretty good, but if you can add more, that would be even better.

In [15]:
len(tokenizer.encode(text_sequence))

KeyboardInterrupt: 

In [22]:
# Sample first 100k characters to estimate total tokens
sample_size = 1000
sample = text_sequence[:sample_size]
tokens_per_char = len(tokenizer.encode(sample)) / len(sample)
estimated_total = int(tokens_per_char * len(text_sequence))
print(f"Estimated total tokens: {estimated_total}")

Estimated total tokens: 823008


Save the tokenizer

In [20]:
import pickle
from pathlib import Path

# Create the directory structure
save_dir = Path("model_llm/output/tokenizer")
save_dir.mkdir(parents=True, exist_ok=True)

# Save tokenizer
with open(save_dir / "my_tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)